# Motivation

2019 GOMS 설문조사를 바탕사회 초년생의 이직의도에 영향을 미치는 요인 파악

# Loading Library

In [7]:
#!pip install brewer2mpl
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; 
warnings.filterwarnings(action='ignore')


# warning 삭제용
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

## Loading Data

In [133]:
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../src')
from data import *



In [134]:
# data.py
df

,g181d001,g181d006,g181e001,g181pid,g181majorcat,g181sex,g181birthy,g181age,g181graduy,g181a001,...,g181q021,g181q022,g181q023,g181p001,g181p008,g181p036,g181p046,g181p041,worker_type,regular_worker
3,NaN,NaN,2.0,100034,1,2,1994,25.583333,2018,2018.0,...,2,5,1,1,2,5,1,2,employed,yes
4,NaN,NaN,2.0,100036,1,1,1997,21.750000,2018,2018.0,...,4,3,4,1,2,4,1,2,employed,yes
5,NaN,NaN,2.0,100047,1,1,1994,25.250000,2018,2018.0,...,4,4,4,1,2,3,1,2,employed,yes
6,1.0,2.0,2.0,100049,1,1,1995,24.500000,2018,2018.0,...,6,4,2,1,2,4,1,2,employed,yes
7,NaN,NaN,2.0,100061,1,1,1993,26.000000,2018,2018.0,...,4,1,4,1,2,4,3,2,employed,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18152,NaN,NaN,2.0,913797,7,2,1993,25.833333,2017,2016.0,...,4,4,4,1,2,5,1,2,employed,yes
18156,NaN,NaN,2.0,913858,7,2,1995,24.000000,2018,2018.0,...,3,2,3,1,2,2,1,2,employed,yes
18157,NaN,NaN,2.0,913874,7,2,1993,25.833333,2017,2018.0,...,2,2,1,1,2,4,1,2,employed,yes
18159,NaN,NaN,2.0,913907,7,2,1994,25.083333,2018,2018.0,...,4,4,5,1,2,3,1,2,employed,yes


In [6]:
#import seaborn as sns
#sns.get_dataset_names()
#df = sns.load_dataset("titanic")
#df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [135]:
# train test split
from sklearn.model_selection import train_test_split

target = "g181a297"
#target = "survived"

def split(df, target, test_size=0.3):
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split(df, target)
    

In [136]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from category_encoders import OrdinalEncoder
# column tranformer
from sklearn.compose import ColumnTransformer

In [147]:
feature_dict =  {
      'g181pid':'id', # id 
       'g181majorcat':'majorcat_cat', # 전공계열
       'g181sex':'sex_cat', # 성별   
       'g181birthy':'birth_date', # 출생년
       'g181age':'age_num', # 연령
       'g181graduy':'graduy_date', #졸업년
### 사업체 관련       
        'g181a001':'year_start_date', # 현일자리 시작년
        'g181a002':'month_start_date', # 현일자리 시작월
        'g181a004_10':'ind_cat', # 일자리 산업 대분류
        'g181a010':'corp_worker_cat', # 기업체 종사자 수 # 결측값문제로 categorical로 변경
        'g181a011':'biz_worker_cat', # 사업체 종사자 수
        'g181a018':'tw_hour', # 출근시간_시간
        'g181a019':'tw_min', # 출근시간_분
        'g181a116':'workday_num', # 주당 정규 근로일
        'g181a117':'worktime_num', # 주당정규 근로시간
        'g181a118':'worktime_ex_num', # 주당초과 근로시간
        'g181a119': 'holy_work_num', # 월평균 휴일근로
        'g181a020': 'biztype_cat', # 사업체형태
       'g181a022':'regular_cat',     # 정규직 비정규직 여부
### 혜택관련
        'g181a390':'voluntary_cat',     # 일자리 형태 자발, 비자발 여부
        'g181a035':'shift_cat' ,    # 교대제 여부
        'g181a038':'pension_cat',     # 퇴직금 제공 여부
        'g181a039':'payed_vc_cat',     # 제공여부 2- 유급휴가
        'g181a043':'maternity_cat',     #  6- 육아휴직
        'g181a045':'overtime_pay_cat',     # 8- 시간 외 수당
        'g181a046':'bonus_cat',     # 9- 상여금
        'g181a048':'weekly_hl_cat',     # 11- 유급주휴
        'g181a392':'baby_vc_cat',     # 12- 산전후휴가
        'g181a120':'wage_type_cat',     # 급여 형태 구분
        'g181a122':'month_wage_num',     # 월 평균 근로소득
        'g181a126':'sat_wage_num',     # 만족도-임금
        'g181a127':'sat_stable_num',     # 만족도-고용안정성
        'g181a128':'sat_work_num',     # 만족도-직무내용
        'g181a129':'sat_env_num',     # 만족도-근무환경
        'g181a130':'sat_wt_num',     # 만족도-노동시간
        'g181a131':'sat_potential_num',     # 만족도-발전가능성
        'g181a132':'sat_relation_num',     # 만족도-인간관계
        'g181a133':'sat_welfare_num',     # 만족도-복리후생
        'g181a134':'sat_hr_num',     # 만족도-인사체계
        'g181a135':'sat_rep1_num',     # 만족도-사회적평판-일
        'g181a136':'sat_auto_num',     # 만족도-자율성 및 권한
        'g181a137':'sat_rep2_num',     # 만족도-일자리-사회적 평판
        'g181a138':'sat_fit_num',     # 만족도-적성흥미일치도
        'g181a139':'sat_edu_num',     # 만족도-직무관련 교육
        'g181a140':'sat_general_num',     # 만족도-일자리_전반적만족도
        'g181a141':'sat_work-general_num',     # 만족도-업무_전반적만족도
        'g181a142':'edu-fit_num',     # 교육수준-일수준일치정도
        'g181a143':'skill-fit_num',     # 일기술수준-본인기술수준일치정도
        'g181a144':'major-fit_num',     # 주전공일치정도
        'g181a146':'major_help_num',     # 전공지식업무도움정도
        'g181a158':'ins_1_num',     # 보험-국민연금
        'g181a159':'ins_2_num',     # 보험-특수직역연금
        'g181a160':'ins_3_num',     # 보험-건강보험
        'g181a161':'ins_4_num',     # 보험-고용보험
        'g181a162':'ins_5_num',     # 보험-산재보험
        'g181a189':'seeking_time_num',     # 구직활동경험기간-개월
        'g181a283':'adjust_difficulty_cat',     # 다른일자리제의여부
        'g181a285':'job_offer_cat',     # 적응시어려움여부
        'g181a297':'turnover_intention',     # 이직준비 여부: target
        'g181g001':'graduate_cat',     # 대학원 경험유무
        'g181l001':'train_cat',     # 취업훈련경험유무
        'g181q001':'health_num',     # 현재 견강상태
        'g181q004':'smoke_cat',     # 흡연여부
        'g181q006':'drink_num,',     # 음주빈도
        'g181q015':'lifesat_personal',     # 삶의만족도-개인적 측면
        'g181q016':'lifesat_relational',     # 삶의만족도-관계적 측면
        'g181q017':'lifesat_group',     # 삶의만족도-소속집단
        'g181q018':'emg_joy_num',     # 감정빈도-즐거운
        'g181q019':'emg_happy_num',     # 감정빈도-행복한
        'g181q020':'emg_comfort_num',     # 감정빈도-편안한
        'g181q021':'emb_irr_num',     # 감정빈도-짜증나는
        'g181q022':'emb_negative_num',     # 감정빈도-부정적인
        'g181q023':'emb_spiritless',     # 감정빈도-무기력한
        'g181p001':'marriage_cat',     # 혼인여부
        'g181p008':'child_cat',     # 부양자녀 유무
        'g181p036':'parent_asset_cat',     # 부모님 자산규모
        'g181p046':'livetype_cat',     # 거주형태
        'g181p041':'support_cat',      # 경제적 지원여부
        # subset에서 추가한 변수들
        'worker_type':'worker_type_cat',
        'regular_worker':'regular_worker_cat',
        'turnover_exp':'turnover_exp_cat',
        'work_exp':'work_exp_cat'        
}


import numpy as np

class Preprocess():
    """
    X_train, X_test
    """
    
    def __init__(self):
        """
        
        """
        print("Preprocessing Class")
        self.numeric_df = None
        self.cat_df = None
        self.ordinal_col = ['biz_worker_cat','parent_asset_cat','turnover_exp_cat']
        
        
    
    def get_dtypes(self,data):
        # onehot 할 피처
        #self.numeric_df = data.select_dtypes(include=['number'])
        #self.cat_df = data.select_dtypes(exclude=['number'])
        self.cat_col = [col for col in data.columns.tolist() if col.endswith('cat') ] 
        self.onehot_col = [col for col in self.cat_col if col not in self.ordinal_col]
        self.numeric_col = [col for col in data if col not in self.onehot_col and col not in self.ordinal_col]
        
                 
    def engineer(self,data):
        
        # id 칼럼 제거
        data = data.drop(['g181pid'], axis=1)
    
        # 이직경험 빈도 추가
        turnover_cond = [
        (data['g181d001'].isnull()==True ), # 첫 직장 전 취업한적 없음(알바포함)
        (data['g181d001']==1) & (data['g181d006']==1), # 알바 경험 있음
        (data['g181d001']==1) & (data['g181d006']==2) & (data['g181e001']==2), # 전직장 있음, 1번이직
        (data['g181e001']== 1)] # 전직장 2개 이상 있음
        choices = [0, 1, 2, 3]

        data['turnover_exp'] = np.select(turnover_cond, choices,default=3)
    
        data.drop(['g181d001','g181d006','g181e001'], axis=1, inplace=True)

        # 알바이외 근로경험 여부
        data['work_exp'] = np.where(data['turnover_exp'].isin([0,1]),0,1)
    
        # 구직활동기간 결측값 0으로 처리
    
        data['g181a189'] = data['g181a189'].fillna(0)
    
        # feature name 변경 (map)
    
        data.columns = data.columns.map(feature_dict)
    
    
        data = data.drop(data[data['month_wage_num'] == -1].index) # 급여 모르는 경우 제거
    
        data['work_year'] = 2019 - data['year_start_date']  # 근무기간
    
        data['work_time_num'] = data['tw_min'] + data['tw_hour'] # 출근소요시간
    
        # 보험 수
        insurances_col = [col for col in data if col.startswith('ins')]
        data['insurances_num'] = 0
        for col in insurances_col:
            data['temp'] = np.where(data[col] == 1, 1, 0)
            data['insurances_num'] += data['temp']
    
        # 회사 전반적 만족도
        biz_sat_col = [col for col in data if col.startswith('sat')]
        data['biz_sat'] = data[biz_sat_col].sum(axis=1)
    
        # 긍정적 감정
        pos_col = [col for col in data if col.startswith('emg') ] 
        data['pos'] = data[pos_col].sum(axis=1)
    
        # 부정적 감정
        neg_col = [col for col in data if col.startswith('neg') ]
        data['neg'] = data[neg_col].sum(axis=1)
    
        # 삶의 만족도
        lifesat_col = [col for col in data if col.startswith('lifesat')]
        data['lifesat'] = data[lifesat_col].sum(axis=1)
        
        # 혜택 수
        benefit_col  =['pension_cat',     # 퇴직금 제공 여부
            'payed_vc_cat',     # 제공여부 2- 유급휴가
            'maternity_cat',     #  6- 육아휴직
            'overtime_pay_cat',     # 8- 시간 외 수당
            'bonus_cat',     # 9- 상여금
            'weekly_hl_cat',     # 11- 유급주휴
            'baby_vc_cat'] # 출산휴가

        data['benefit_num'] = 0

        for cols in benefit_col:
            data['temp'] = np.where(data[cols]==1, 1, 0)
            data['benefit_num'] += data['temp']
        
        # 결측값 0으로 처리(구직활동기간이 0이기에)    
        data['seeking_time_num'] = data['seeking_time_num'].fillna(0)
    
        data.drop(columns='temp',axis=1,inplace=True) # temp 삭제
        data.drop(data[data.month_wage_num==-1].index,inplace=True) # 급여 모르는 경우 제거
    
        return data
    
    def drop_col(self,data,col):
        """
        추가적인 가설검증이나 모델링 결과에 따라 드롭해야할 컬럼이 생길 경우 사용
        """
        data.drop(columns=col,axis=1,inplace=True)
        
        return data
        
    
    def base_pipline(self,data):
        
      
        """

        """
        
        cat_pipeline = Pipeline(steps=[
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('one-hot', OneHotEncoder(use_cat_names=True,cols=self.onehot_col)),
            ('ordial', OrdinalEncoder(cols=self.ordinal_col))
        ])
        num_pipeline = Pipeline(steps=[
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('scale', StandardScaler())
        ])
        preprocess_pipeline = ColumnTransformer(
            transformers=[
                ('num', num_pipeline, self.numeric_col),
                ('cat', cat_pipeline, self.cat_col)
            ]
        )
        data = preprocess_pipeline.fit_transform(data)
        return data


class PreprocessSelector():
    """
    전처리 방식 선택
    
    """
    def __init__(self):
        self.data=None
        self._preprocessor=Preprocess()

    def strategy(self, data, strategy_type="strategy1"):
        self.data=data
        if strategy_type=='strategy1':
            self._strategy1()
        elif strategy_type=='strategy2':
            self._strategy2()

        return self.data

    def _base_strategy(self):
        
        self.data=self._preprocessor.engineer(self.data)
        self._preprocessor.get_dtypes(self.data)
        self.data=self._preprocessor.base_pipline(self.data)

    def _strategy1(self):

        self._base_strategy()
        self.data = self._preprocessor.drop_col(self.data,"corp_worker_cat")
    

    def _strategy2(self):
        self._base_strategy()

        self.data=self._preprocessor._get_dummies(self.data,
                                        prefered_columns=None)#None mean that all feature will be dummied

In [148]:
temp_selector=PreprocessSelector()

Preprocessing Class


In [149]:
temp_selector.data = X_train

In [150]:
temp_selector.data = temp_selector._preprocessor.engineer(temp_selector.data)


In [151]:
temp_selector._preprocessor.get_dtypes(temp_selector.data)

In [152]:
temp_selector.data

,majorcat_cat,sex_cat,birth_date,age_num,graduy_date,year_start_date,month_start_date,ind_cat,corp_worker_cat,biz_worker_cat,...,turnover_exp_cat,work_exp_cat,work_year,work_time_num,insurances_num,biz_sat,pos,neg,lifesat,benefit_num
712,1,1,1990,29.166667,2018,2015.0,10.0,15.0,9.0,5.0,...,0,0,4.0,40.0,4,64.0,14,0.0,18,6
6800,4,1,1993,26.166667,2018,2017.0,12.0,8.0,9.0,7.0,...,0,0,2.0,10.0,4,80.0,21,0.0,21,7
12915,6,1,1994,24.833333,2018,2019.0,4.0,3.0,7.0,3.0,...,2,1,0.0,30.0,3,56.0,12,0.0,21,7
11758,4,1,1992,26.833333,2018,2018.0,4.0,3.0,4.0,4.0,...,0,0,1.0,1.0,4,66.0,21,0.0,18,6
13237,1,1,1991,28.500000,2018,2018.0,1.0,15.0,-1.0,4.0,...,0,0,1.0,25.0,3,61.0,14,0.0,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,6,1,1993,26.083333,2018,2018.0,4.0,17.0,-1.0,1.0,...,0,0,1.0,1.0,4,67.0,18,0.0,18,7
9122,4,1,1992,26.750000,2018,2018.0,1.0,13.0,8.0,8.0,...,0,0,1.0,30.0,4,48.0,12,0.0,13,4
9404,4,1,1992,27.416667,2018,2018.0,1.0,17.0,6.0,6.0,...,0,0,1.0,15.0,4,54.0,12,0.0,12,7
1507,3,2,1994,25.500000,2018,2018.0,3.0,16.0,5.0,5.0,...,0,0,1.0,20.0,2,79.0,21,0.0,21,6


In [165]:
x_data = temp_selector.data
ordinal_col = ['biz_worker_cat','parent_asset_cat','turnover_exp_cat']
cat_col = [col for col in x_data.columns.tolist() if col.endswith('cat') ] 
onehot_col = [col for col in cat_col if col not in ordinal_col]
numeric_col = [col for col in x_data.columns.tolist() if col not in onehot_col and col not in ordinal_col]

    
"""     
cat_pipeline = Pipeline(steps=[
            ('one-hot', OneHotEncoder(use_cat_names=True,cols=onehot_col)),
            ('ordinal', OrdinalEncoder(cols=ordinal_col))
])
"""

num_pipeline = Pipeline(steps=[
            ('impute', SimpleImputer()),
            ('scale', StandardScaler())])

preprocess_pipeline = ColumnTransformer(
            transformers=[
                ('num', num_pipeline, numeric_col),
                ('ord',)
                ('ohe', cat_pipeline, onehot_col)
            ]
)


In [ ]:
"""


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
numeric_features = numeric_col 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = category_col
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
clf = Pipeline(steps=[('preprocessor', preprocessor),])

"""


In [164]:
cat_pipeline.fit_transform(x_data)

KeyError: 'majorcat_cat'

In [158]:
x_data['majorcat_cat']

712      1
6800     4
12915    6
11758    4
13237    1
        ..
9971     6
9122     4
9404     4
1507     3
12567    7
Name: majorcat_cat, Length: 7367, dtype: int64

In [156]:
temp_pipline(x_data)

KeyError: 'majorcat_cat'

In [97]:
temp_selector._preprocessor.cat_df

In [144]:
temp_selector.data = temp_selector._preprocessor.base_pipline(temp_selector.data)

ValueError: could not convert string to float: 'employed'

- feature selection과 classification method의 조합
- hyperparameter tuning은 greed search cv
- permutation importance의 경우 변수간 상관관계를 고려할 필요요가 있음
  
- feauture의 수를 줄일 것
- 다중공선성 검정
